## 기본설정 및 전처리

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.offline as offline
from folium.plugins import HeatMapWithTime
from plotly.subplots import make_subplots
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium import FeatureGroup
import json
import math
import re
from datetime import datetime
import os
import glob
import subprocess
from bs4 import BeautifulSoup as bs
from shapely.geometry import Point, Polygon, LineString
import geopandas as gpd
from geopandas import GeoSeries
import pyproj
from tqdm import tqdm
#from keplergl import KeplerGl

# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

In [2]:
# Polygon을 만드는 함수
def make_pol(x):
    try:
        return Polygon(x[0])
    except:
        return Polygon(x[0][0])

In [3]:
# Linestring을 만드는 함수
def make_lin(x):
    try:
        return LineString(x)
    except:
        return LineString(x[0])

In [4]:
# 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
def geo_transform(DataFrame) :
    # csv to geopandas
    # lon, lat data를 geometry로 변경
    DataFrame['lat'] = DataFrame['lat'].astype(float)
    DataFrame['lon'] = DataFrame['lon'].astype(float)
    DataFrame['geometry'] = DataFrame.apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
    DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
    DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
    DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
    return DataFrame

### 청주시_도시재생계획구역

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/27.청주시_도시재생계획구역.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
crp_df = pd.json_normalize(geojson_data) # city_revitalize_planning_map_df
crp_df['geometry'] = crp_df['geometry.coordinates'].apply(lambda x : make_pol(x))
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)

# 폴리곤의 중점을 찾음
centroid = crp_df['geometry'].iloc[0].centroid.buffer(0.015) # 1도의 위도 변화는 대략 111.32 킬로미터
crp_df['centroid_polygon_geometry'] = [centroid]

### 격자(매핑용)

In [6]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/26.청주시_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))

In [7]:
# crp_df에서 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
polygon_expanded = crp_df['centroid_polygon_geometry'].iloc[0]

# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')
#grid_map_df['geometry'] = GeoSeries(grid_map_df['geometry'])

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid = grid_map_df[grid_map_df['geometry'].intersects(polygon)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택, within 은 포함
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_grid_expanded = grid_map_df[grid_map_df['geometry'].intersects(polygon_expanded)].reset_index(drop=True) # polygon과 교차하거나 포함하는 경우를 모두 선택

#### 청주시 상세 도로망 & 도로명주소 데이터 필터링 및 통합

In [52]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/13.청주시_상세도로망.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadsystem_df = pd.json_normalize(geojson_data['features'])
roadsystem_df['geometry'] = roadsystem_df['geometry.coordinates'].apply(lambda x : LineString(x))
roadsystem_df['properties.link_id'] = roadsystem_df['properties.link_id'].astype(str)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadsystem_df = gpd.GeoDataFrame(roadsystem_df, geometry='geometry')
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadsystem = roadsystem_df[roadsystem_df['geometry'].intersects(polygon)].reset_index(drop=True)

# GeoJSON 파일 불러오기
with open('SBJ_2309_001/16.청주시_도로명주소(도로).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
roadname_road_df = pd.json_normalize(geojson_data['features'])
roadname_road_df['geometry'] = roadname_road_df['geometry.coordinates'].apply(lambda x: LineString(x))
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
roadname_road_df = gpd.GeoDataFrame(roadname_road_df, geometry='geometry')
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_roadname_road = roadname_road_df[roadname_road_df['geometry'].within(polygon)].reset_index(drop=True)

# 통합 도로망 df
tot_roadsystem = pd.concat([filtered_roadsystem[['geometry']], filtered_roadname_road[['geometry']]])

#### 청주시 시장현황 필터링

In [9]:
# 시장 데이터프레임을 GeoPandas 데이터프레임으로 변환
market_df = pd.read_csv('SBJ_2309_001/6.청주시_시장현황.csv')
market_df = geo_transform(market_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# market_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_market = market_df[market_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 청주시 주차장 현황

In [10]:
# 시장 데이터프레임을 GeoPandas 데이터프레임으로 변환
parking_df = pd.read_csv('SBJ_2309_001/7.청주시_주차장현황.csv')
parking_df = geo_transform(parking_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0].centroid.buffer(0.01)
# parking_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_parking = parking_df[parking_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 청주시 공공기관 및 주요지점 현황

In [11]:
public_df = pd.read_csv('SBJ_2309_001/25.청주시_공공기관_및_주요지점현황.csv')
# 데이터프레임을 GeoPandas 데이터프레임으로 변환
public_df = geo_transform(public_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# public_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_public = public_df[public_df['geometry'].within(polygon)]
filtered_points_public['inst_gbn'] = filtered_points_public['inst_gbn'].replace('공공기관', '행정기관')
filtered_points_public['inst_gbn'] = filtered_points_public['inst_gbn'].replace('시군청', '행정기관')
filtered_points_public['inst_gbn'] = filtered_points_public['inst_gbn'].replace('도단위기관', '행정기관')
filtered_points_public['inst_gbn'] = filtered_points_public['inst_gbn'].replace('민간기관', '상업시설')
filtered_points_public = filtered_points_public.drop_duplicates(subset='geometry', keep='first').reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

#### 청주시 공원 현황

In [12]:
# 시장 데이터프레임을 GeoPandas 데이터프레임으로 변환
park_df = pd.read_csv('SBJ_2309_001/8.청주시_공원현황.csv')
park_df = geo_transform(park_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# park_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_park = park_df[park_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 청주시 인도(보도) 시각화

In [53]:
# GeoJSON 파일 불러오기
with open('SBJ_2309_001/17.청주시_인도(보도).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
sidewalk_df = pd.json_normalize(geojson_data['features'])
sidewalk_df['geometry'] = sidewalk_df['geometry.coordinates'].apply(make_lin)
# roadsystem_df 데이터프레임을 GeoDataFrame으로 변환
sidewalk_df = gpd.GeoDataFrame(sidewalk_df, geometry='geometry')
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_sidewalk = sidewalk_df[sidewalk_df['geometry'].intersects(polygon)].reset_index(drop=True)

#### 청주시 폐공장현황

In [14]:
# 데이터프레임 로드 및 GeoPandas 데이터프레임으로 변환
factory_df = pd.read_csv('SBJ_2309_001/9.청주시_폐공장현황.csv')
factory_df = geo_transform(factory_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_factory = factory_df[factory_df['geometry'].within(polygon)]

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 청주시 문화재 현황

In [15]:
# 데이터프레임 로드 및 GeoPandas 데이터프레임으로 변환
culture_assets_df = pd.read_csv('SBJ_2309_001/5.청주시_유적지_문화재현황.csv')
culture_assets_df = geo_transform(culture_assets_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

# 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_culture = culture_assets_df[culture_assets_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 청주시 학교 현황

In [16]:
# 데이터프레임 로드 및 GeoPandas 데이터프레임 변환
school_df = pd.read_csv('SBJ_2309_001/24.청주시_학교현황.csv')
school_df = geo_transform(school_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry
# 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_school = school_df[school_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 청주시 어린이집 유치원 현황

In [17]:
# 데이터프레임 로드 및 GeoPandas 데이터프레임으로 변환
kinder_df = pd.read_csv('SBJ_2309_001/28.청주시_어린이집_유치원현황.csv')
kinder_df = geo_transform(kinder_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry
# 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_kinder = kinder_df[kinder_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 청주시 상권정보 

In [18]:
# 상권정보 데이터 로드 및 GeoDataFrame으로 변환
trading_area_df = pd.read_csv('SBJ_2309_001/18.청주시_상권정보.csv')
trading_area_df = geo_transform(trading_area_df)
# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]
# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_trading_area = trading_area_df[trading_area_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 청주시 가로수 현황 (공공데이터포털 - 산림청_도시숲가로수관리 가로수 현황 https://www.data.go.kr/data/15120900/fileData.do)

In [19]:
'''산림청_도시숲가로수관리 가로수 현황_20221231 데이터 정제 완료 -'''
# street_tree_df = pd.read_csv('SBJ_2309_001/산림청_도시숲가로수관리 가로수 현황_20221231.csv', encoding='cp949')
# street_tree_df = street_tree_df[['시군구명', '도로구간명', '수종명', '지역X좌표', '지역Y좌표']]
# street_tree_df = street_tree_df[street_tree_df['시군구명'].str.contains('청주')].reset_index(drop=True)

# # EPSG 5186에서 EPSG 4326으로 좌표 변환을 수행하는 함수 생성
# def transform_coordinates(row):
#     in_proj = pyproj.Proj(init='epsg:5186')
#     out_proj = pyproj.Proj(init='epsg:4326')
#     lon, lat = pyproj.transform(in_proj, out_proj, row['지역X좌표'], row['지역Y좌표'])
#     return pd.Series({'lat': lat, 'lon': lon})

# # '지역X좌표'와 '지역Y좌표'를 EPSG 4326으로 변환하고 'lat' 및 'lon' 열로 이름 변경
# street_tree_df[['lat', 'lon']] = street_tree_df.apply(transform_coordinates, axis=1)
# # '지역X좌표'와 '지역Y좌표' 열을 삭제 (선택 사항)
# street_tree_df.drop(['지역X좌표', '지역Y좌표'], axis=1, inplace=True)

# 가로수 데이터프레임 로드 및 GeoPandas 변환
street_tree_df = pd.read_csv('SBJ_2309_001/29.청주시_가로수현황.csv', encoding='UTF8')
street_tree_df = geo_transform(street_tree_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # 또는 centroid_polygon_geometry

# factory_df의 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_points_tree = street_tree_df[street_tree_df['geometry'].within(polygon)].reset_index(drop=True)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



## 버스정류장 현황 및 승하차이력

In [20]:
# 승하차data load 및 전처리(통합)
bususe_df = pd.read_csv('SBJ_2309_001/11.청주시_버스정류장_승하차이력.csv')
bususe_df['tot_use'] = bususe_df.iloc[:,6:10].sum(axis=1)
bususe_df_lit = bususe_df[['bis_id', 'stn_nm', 'board_gbn' ,'tot_use']].copy()
bususe_df_tot = bususe_df_lit.groupby(['bis_id', 'stn_nm', 'board_gbn'])['tot_use'].sum().reset_index()

# 승하차 유형을 통합한 bususe_df_unique_tot 정의 & bis_id와 승하차량을 매핑한 bid_use_dic 정의
bususe_df_unique_tot = pd.DataFrame()
for bid in bususe_df_tot['bis_id'].unique():
    subset = bususe_df_tot[bususe_df_tot['bis_id'] == bid]
    tot = subset.iloc[0:1,0:2]
    tot['tot_use'] = subset['tot_use'].sum()
    bususe_df_unique_tot = pd.concat([bususe_df_unique_tot, tot]).reset_index(drop=True)
bid_use_dic = dict(zip(bususe_df_unique_tot['bis_id'], bususe_df_unique_tot['tot_use']))

In [51]:
busstop_df = pd.read_csv('SBJ_2309_001/10.청주시_버스정류장현황.csv')
busstop_df = geo_transform(busstop_df)

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0] # buffer로 원도심 centoid에서 약 2.3km반경 영역

# 'geometry' 열을 사용하여 Point 객체를 필터링
filtered_busstop = busstop_df[busstop_df['geometry'].within(polygon)].reset_index(drop=True)

# bid_use_dic을 이용하여 정류장의 승하차량 'tot_use' column 생성
filtered_busstop['tot_use'] = filtered_busstop['bis_id'].map(bid_use_dic)
filtered_busstop.dropna(inplace=True)
filtered_busstop['tot_use'] = filtered_busstop['tot_use'].astype(int)

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

c:\Users\user\anaconda3\envs\GIS\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6



#### 청주시 추정교통량

In [22]:
# 도로 id와 geometry를 매핑한 딕셔너리 생성 : 추정교통량 data와 매핑 예정
road_geo_dic = dict(zip(roadsystem_df['properties.link_id'], roadsystem_df['geometry']))

traffic_df = pd.read_csv('SBJ_2309_001/12.청주시_추정교통량.csv')
traffic_df.rename(columns={'year' : '연도'}, inplace=True)
traffic_df['연도'] = traffic_df['연도'].astype(str)
traffic_df['link_id'] = traffic_df['link_id'].astype(str)
traffic_df['link_id'] = traffic_df['link_id'].apply(lambda x : x[:-2])
traffic_df

C:\Users\user\AppData\Local\Temp\ipykernel_6108\3444191130.py:4: DtypeWarning:

Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.



,연도,link_id,road_rank,road_length,road_nm,sido_nm,sgg_nm,emd_nm,timeslot,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT
0,2019,900000055,107,0.327926,NaN,충청북도,청주시 서원구,현도면,all,1198,136,24,1038
1,2019,900000055,107,0.327926,NaN,충청북도,청주시 서원구,현도면,all,1531,126,24,1381
2,2019,900000070,107,0.341045,NaN,충청북도,청주시 서원구,현도면,all,1076,126,24,925
3,2019,900000070,107,0.341045,NaN,충청북도,청주시 서원구,현도면,all,1268,119,25,1124
4,2019,564304590,101,0.253755,당진영덕고속도로,충청북도,청주시 상당구,문의면,all,2457,351,85,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1340495,2021,563413568,108,0.256000,경부고속도로,충청북도,청주시 흥덕구,옥산면,9,234,210,8,17
1340496,2021,563413564,108,0.249000,경부고속도로,충청북도,청주시 흥덕구,옥산면,9,165,97,13,54
1340497,2021,563506287,108,0.926000,경부고속도로,충청북도,청주시 흥덕구,옥산면,9,503,354,14,135
1340498,2021,563506038,108,1.456000,경부고속도로,충청북도,청주시 흥덕구,옥산면,9,498,96,14,387


In [23]:
# 딕셔너리 생성 및 연도 & DataFrame 매핑
traffic_df_year_dic = {}
year_list = ['2019', '2020', '2021']

# crp_df에서 첫 번째 폴리곤 영역을 선택 = crp_df는 원도심 영역(도시재생 대상지역)
polygon = crp_df['geometry'].iloc[0]

for year in year_list:
    traffic_df_year = traffic_df[traffic_df['연도'] == year].copy() # 특정 연도로만 필터링
    traffic_df_year['geometry'] = traffic_df_year['link_id'].map(road_geo_dic)
    # polygon을 사용하여 폴리곤 내의 도로망 필터링
    traffic_df_year = gpd.GeoDataFrame(traffic_df_year, geometry='geometry')
    traffic_df_year = traffic_df_year[traffic_df_year['geometry'].intersects(polygon)].reset_index(drop=True)
    traffic_df_year = traffic_df_year[traffic_df_year['timeslot'] == 'all']
    concat_df = pd.DataFrame()
    for road_id in tqdm(traffic_df_year['link_id'].unique()):
        subset = traffic_df_year[traffic_df_year['link_id'] == road_id]
        column_sums = subset.iloc[:,9:13].sum()
        column_sums_df = pd.DataFrame(column_sums).transpose()
        subset_front = pd.DataFrame(subset.iloc[0,:9]).transpose().reset_index(drop=True)
        subset_back = pd.DataFrame(subset.iloc[0,13:]).transpose().reset_index(drop=True)
        re_subset = pd.concat([subset_front, column_sums_df, subset_back], axis=1)
        concat_df = pd.concat([concat_df, re_subset])
    else:
        traffic_df_year_dic[year] = concat_df

100%|██████████| 86/86 [00:00<00:00, 222.63it/s]


#### 연도별 교통량 HEATMAP

In [24]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 통합도로망 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = tot_roadsystem['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    }
)
grid_layer.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2019 = folium.FeatureGroup(name="2019년", overlay=False)
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)

year_lst = [str(year) for year in range(2019, 2022)]
layers_lst = [m_2019, m_2020, m_2021]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2019부터 2021까지의 연도
    subset = traffic_df_year_dic[year]
    # 'ALL_AADT' 열의 값을 교통량으로 사용하고 'geometry' 열의 값을 위치 정보로 사용하여 HeatMap을 생성합니다.
    heat_data = [[point.centroid.xy[1][0], point.centroid.xy[0][0], traffic] for point, traffic in zip(subset['geometry'], subset['ALL_AADT'])]
    HeatMap(heat_data, max_opacity=0.6, radius=17).add_to(layers_dic[year])

for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 지도 출력
m

In [26]:
m.save("visualization/3. 원도심 교통 분석/원도심 교통량 HEATMAP.html")

#### 연도별 도로망 - 교통량 시각화

In [31]:
subset

,연도,link_id,road_rank,road_length,road_nm,sido_nm,sgg_nm,emd_nm,timeslot,ALL_AADT,PSCR_AADT,BUS_AADT,FGCR_AADT,geometry
0,2021,564429120,103,0.012,사직대로,충청북도,청주시 상당구,성안동,all,26239,19141,3448,3649,LINESTRING (127.48627662716854 36.636675377423...
0,2021,564429156,103,0.019,사직대로,충청북도,청주시 상당구,성안동,all,27126,20423,3666,3037,LINESTRING (127.48627662716854 36.636675377423...
0,2021,564407239,103,0.087,상당로,충청북도,청주시 상당구,성안동,all,23911,17425,3531,2955,"LINESTRING (127.49131716652 36.62861533009315,..."
0,2021,564413123,103,0.12,상당로,충청북도,청주시 상당구,성안동,all,24420,17703,3206,3511,LINESTRING (127.49111696582575 36.630478578508...
0,2021,564429155,103,0.126,사직대로,충청북도,청주시 상당구,성안동,all,25203,20135,2482,2586,LINESTRING (127.48466207470291 36.636531270758...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2021,564430292,107,0.178,중앙로,충청북도,청주시 상당구,중앙동,all,3268,2700,238,330,LINESTRING (127.48844929794456 36.642591008416...
0,2021,564416940,107,0.211,교서로,충청북도,청주시 상당구,중앙동,all,9368,8085,409,873,LINESTRING (127.48619443034215 36.640855795835...
0,2021,564404453,107,0.212,교동로3번길,충청북도,청주시 상당구,중앙동,all,7975,6284,785,905,LINESTRING (127.49056610288459 36.641150795316...
0,2021,564400030,107,0.218,교동로3번길,충청북도,청주시 상당구,중앙동,all,2441,1664,406,371,LINESTRING (127.49072640629673 36.637279934244...


In [56]:
#  교통량에 따라 도로망 색상을 지정하는 함수
def pick_color(traffic, dataframe):
    desc_lst = subset.describe().iloc[4:7,0].tolist()
    top_25 = desc_lst[-1]
    top_50 = desc_lst[-2]
    top_75 = desc_lst[-3]
    if traffic > top_25:
        return 'red'
    elif top_25 > traffic >= top_50:
        return 'orange'
    elif top_50 > traffic >= top_75:
        return 'yellow'
    elif top_75 > traffic >= 0:
        return 'green'

# 범례 생성
legend_html = """
     <div style="position: fixed; 
     bottom: 50px; right: 50px; width: 280px; height: 130px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 1);
     "> &nbsp; 교통량 <br>
     &nbsp; <i style="background:red">&nbsp;</i>&nbsp; 상위 0% ~ 25% <br>
     &nbsp; <i style="background:orange">&nbsp;</i>&nbsp; 상위 25% ~ 50% <br>
     &nbsp; <i style="background:yellow">&nbsp;</i>&nbsp; 상위 59% ~ 75% <br>
     &nbsp; <i style="background:green">&nbsp;</i>&nbsp; 상위 75% ~ 100%
     </div>
     """

# map 생성
m = folium.Map(location=[36.627797, 127.511943], zoom_start=14)

# 기본 배경지도를 항상 표시하도록 설정 및 기본 통합 도로망 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
grid_geojson = tot_roadsystem['geometry'].to_json()
grid_layer = folium.GeoJson(
    grid_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 5
    })
grid_layer.add_to(m)

# 인도 추가
m_sidewalk = folium.FeatureGroup(name="인도", overlay=True)
for _, row in filtered_sidewalk.iterrows() :
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature: {
            'fillColor': 'blue',
            'color': 'blue',
            'weight': 1
        }).add_to(m_sidewalk)
m_sidewalk.add_to(m)

# 레이어 추가. 체크해제(비활성화)된 상태로 표시되도록 함
m_2019 = folium.FeatureGroup(name="2019년", overlay=False)
m_2020 = folium.FeatureGroup(name="2020년", overlay=False)
m_2021 = folium.FeatureGroup(name="2021년", overlay=False)

year_lst = [str(year) for year in range(2019, 2022)]
layers_lst = [m_2019, m_2020, m_2021]
layers_dic = {}
for year, layer in zip(year_lst, layers_lst):
    layers_dic[year] = layer

# 연도별 데이터프레임을 만들고 레이어 추가
for year in year_lst:  # 2018부터 2022까지의 연도
    subset = traffic_df_year_dic[year].copy()
    # 'road'(LINESTRING)을 기준으로 그룹화
    for road in tqdm(subset['geometry'].unique()):
        subset_road = subset[subset['geometry'] == road].copy()
        traffic = subset_road['ALL_AADT'].iloc[0]
        color = pick_color(traffic, subset)
        popup_text = f"<div>{subset_road['road_nm']} - {subset_road['ALL_AADT']}</div>"
        folium.GeoJson(
            subset_road['geometry'].iloc[0],
            style_function=lambda feature, color=color: {
                'fillColor': color,
                'color': color,
                'weight': 5}
        ).add_to(layers_dic[year]).add_child(folium.Popup(popup_text, max_width=500), name=str(year))
    
for layer_instance in layers_dic.values():
    layer_instance.add_to(m)
    
# LayerControl을 사용하여 연도 선택
folium.LayerControl(collapsed=False).add_to(m)

# 범례 추가
m.get_root().html.add_child(folium.Element(legend_html))
m

100%|██████████| 86/86 [00:00<00:00, 142.63it/s]


In [57]:
m.save("visualization/3. 원도심 교통 분석/원도심 연도별 도로망-교통량 시각화.html")

#### 통합 MAP 제작

In [58]:
# 지도 생성
m = folium.Map(location=[36.644, 127.485], zoom_start=13)

# 기본 배경지도를 항상 표시하도록 설정 및 도로망 추가
folium.TileLayer('openstreetmap', overlay=False).add_to(m)
roal_geojson = tot_roadsystem['geometry'].to_json()
roal_layer = folium.GeoJson(
    roal_geojson,
    name="도로망",
    style_function=lambda feature: {
        'fillColor': 'none',
        'color': 'gray',
        'weight': 3
    })
roal_layer.add_to(m)

# 인도 추가
m_sidewalk = folium.FeatureGroup(name="인도", overlay=True)
for _, row in filtered_sidewalk.iterrows() :
    folium.GeoJson(
        row['geometry'],
        style_function=lambda feature: {
            'fillColor': 'blue',
            'color': 'blue',
            'weight': 1
        }).add_to(m_sidewalk)
m_sidewalk.add_to(m)

# 'geometry'(위치 정보) & 'tot_use'(사용건수)를 사용하여 HeatMap 생성
heat_data = [[point.xy[1][0], point.xy[0][0], tot_use] for point, tot_use in zip(filtered_busstop['geometry'], filtered_busstop['tot_use'])]
HeatMap(heat_data, max_opacity=0.6, radius=25, name="승하차 히트맵").add_to(m)

m_busstop = folium.FeatureGroup(name="버스정류장", overlay=True)
for _, row in filtered_busstop.iterrows() :
    popup_text = f"{row['stn_nm']} - {row['tot_use']}건"
    folium.Circle(location=(row['lat'], row['lon']), radius=10, color='black',
            fill='#FF580B', name="버스 정류장").add_to(m_busstop).add_child(folium.Popup(popup_text, max_width=200))
m_busstop.add_to(m)

# LayerControl을 사용하여 주중 & 주말 선택할 수 있게 구성
folium.LayerControl(collapsed=False).add_to(m)
# 지도 출력
m

In [59]:
m.save("visualization/3. 원도심 교통 분석/원도심 버스승하차량 HEATMAP.html")